## Async programming
Task asynchronous programming model(TAP).
Task await doesnt block thread.
if any part of code is async entire code block is async- tells compiler that some threads will be released when await. 

In depth: https://docs.microsoft.com/en-us/dotnet/standard/async-in-depth

Important!
when task is I/O bound use async await in standard form: if I/O-bound task is run no thread is dedicated to running the task.
Most of the time application hands off work to other system lib that takes time. thats why servers should use async- it frees 
server pool when I/O task is being done. IT should be done like that is asp net too:
https://stackoverflow.com/questions/41953102/using-async-await-or-task-in-web-api-controller-net-core

In [ ]:
//this starts task 
private bool CalculateDamageDone()
{
    // Code omitted:
    //
    // Does an expensive calculation and returns
    // the result of that calculation.
    return true;
}

var task  = async (o, e) =>
{
    // This line will yield control to the UI while CalculateDamageDone()
    // performs its work. The UI thread is free to perform other work.
    var damageResult = await Task.Run(() => CalculateDamageDone());
    // DisplayDamage(damageResult);
};

This is because it spawns another thread. Keep in mind this does not offer any extra protection for shared data.
You should use Task.Run, but not within any code you want to be reusable (i.e., library code). So you use Task.Run to call the method, not as part of the implementation of the method.
Async void should only be used for event handlers. On ui exceptions are thrown onto dispatcher. Normaly they cant be awaited on
and exception crashes app.
Tread carefully when using async lambdas in LINQ expressions

returning reference type may impact performance. Consider using ValueTask from c#7.0. You may define your own return type with gGetAwaiter method to return task.
IAsyncEnumerable works for async methods that return streams from C#8.0

### Cancelling tasks

W celu przerywania taskow uzywa sie CancellationTokenSource. Podaje sie taki obiekt w parametrze do kazdeego taska.

### Obslugiwanie taskow w miare ich ukonczenia
metoda ta jest zlozona obliczeniowa w przypadku duzej ilosci taskow powinno sie uzywac metod opisanych w linku ponizej

In [ ]:


while (downloadTasks.Any())
    {
        Task<int> finishedTask = await Task.WhenAny(downloadTasks);
        downloadTasks.Remove(finishedTask);
        total += await finishedTask;
    }

This works for small number of tasks, for more see https://devblogs.microsoft.com/pfxteam/processing-tasks-as-they-complete/
since it creates N^2 algorithm for no reason.

# exceptions for tasks

Tasks that throws is called faulted.
Tasks hold exceptions thrown in Task.Exception property as System.AggregateException.
Exception is thrown when task is awaited on and first exception from AggregateException.InnerExceptions is thrown.

Task logic: 
```
await Task.WhenAll(eggsTask, baconTask, toastTask);
Task finishedTask = await Task.WhenAny(breakfastTasks);
```


# Atrybuty
Atrybuty moga byc dodawane w runtime za pomoca refleksji. Konwencja: nazwy atrybutów kończą się zawsze na Attribute,


In [ ]:
// przyklad uzycia atrybutow dla parametru
void MethodA([In][Out] ref double x) { }
void MethodB([Out][In] ref double x) { }
void MethodC([In, Out] ref double x) { }

Attribute target: target to entity ktore atrybut aplikuje sie do.
Aby jawnie okreslic cel atrybutu nalezy uzyc skladni

In [ ]:
[target : attribute-list]

Target value	Applies to

assembly	Entire assembly

module	Current assembly module

field	Field in a class or a struct

event	Event

method	Method or get and set property accessors

param	Method parameters or set property accessor parameters

property	Property

return	Return value of a method, property indexer, or get property accessor

type	Struct, class, interface, enum, or delegate

In [ ]:
// default: applies to method
[ValidatedContract]
int Method1() { return 0; }

// applies to method
[method: ValidatedContract]
int Method2() { return 0; }

// applies to parameter
int Method3([ValidatedContract] string contract) { return 0; }

// applies to return value
[return: ValidatedContract]
int Method4() { return 0; }

Popularne użycia atrybutów:
- Marshalling interopów 
- Określanie właściwosci COM (Component Object Model- windowsowy stuff)
- Dll importy
- Okreslanie pol do serializacji

Definiowanie wlasnego atrybutu z refleksja do jego odczytania:

In [ ]:
// Multiuse attribute.  
[System.AttributeUsage(System.AttributeTargets.Class |  
                       System.AttributeTargets.Struct,  
                       AllowMultiple = true)  // Multiuse attribute.  
]  
public class Author : System.Attribute  
{  
    string name;  
    public double version;  
  
    public Author(string name)  
    {  
        this.name = name;  
  
        // Default value.  
        version = 1.0;  
    }  
  
    public string GetName()  
    {  
        return name;  
    }  
}  
  
// Class with the Author attribute.  
[Author("P. Ackerman")]  
public class FirstClass  
{  
    // ...  
}  
  
// Class without the Author attribute.  
public class SecondClass  
{  
    // ...  
}  
  
// Class with multiple Author attributes.  
[Author("P. Ackerman"), Author("R. Koch", version = 2.0)]  
public class ThirdClass  
{  
    // ...  
}  
  
class TestAuthorAttribute  
{  
    static void Test()  
    {  
        PrintAuthorInfo(typeof(FirstClass));  
        PrintAuthorInfo(typeof(SecondClass));  
        PrintAuthorInfo(typeof(ThirdClass));  
    }  
  
    private static void PrintAuthorInfo(System.Type t)  
    {  
        System.Console.WriteLine("Author information for {0}", t);  
  
        // Using reflection.  
        System.Attribute[] attrs = System.Attribute.GetCustomAttributes(t);  // Reflection.  
  
        // Displaying output.  
        foreach (System.Attribute attr in attrs)  
        {  
            if (attr is Author)  
            {  
                Author a = (Author)attr;  
                System.Console.WriteLine("   {0}, version {1:f}", a.GetName(), a.version);  
            }  
        }  
    }  
}  
/* Output:  
    Author information for FirstClass  
       P. Ackerman, version 1.00  
    Author information for SecondClass  
    Author information for ThirdClass  
       R. Koch, version 2.00  
       P. Ackerman, version 1.00  
*/

### Collections
Są dwa generalne podejscia uzycie arraya lub kolekcji.
Arraya uzywamy gdy mamy do cyznienia ze stała ilością mocno typowanych obiektów.

Kolekcje sa bardziej elastyczne potrafia zmieniac swoj rozmiar dynamicznie. Dla kolekcji obiektow o jednym typie uzywamy System.Collections.Generic.

Do System.Collection.Generic zaliczamy 
- Dictionary<TKey, TValue>- key value pairs 
- List<T>- no lista no
- Queue<T>- FIFO(first in first out)
- SortedList<TKey, TValue>- kolekcja par key/value posegregowana wedlug implementacji IComparer<T>
- Stack<T>- last in first out (LIFO) 

System.Collections.Concurent classes- klasy dające watkowe bezpieczne kolekcje.  
Przykladowymi sa:
- BlockingCollection<T>,
- ConcurrentDictionary<TK,TV>, 
- ConcurrentStack<T>

System.Collections classes- kolekcje nie przechowujące typowanych obiektów- obiekty w nich są traktowane jako Object.
- ArrayList reprezentuje arraya ktorego wielkosc jest dynamicznie zwiekszana w razie potrzeby
- Hashtable reprezentuje kolekcje key/value ktora jest segregowana wedlug kodu hash klucza zmienne przechowywane są w bucketach
- Queue FIFO
- stack LIFO

Jezeli obiekt implementuje ICompare mozna jego liste segregowac.

Tworzenie wlasnej kolekcji zachodzi przez imlementacje **IEnumerable**. Wymaga to zaimplementowania enumeratora i metody **GetEnumerator** w klasie koleksji. Enumerator ma current i move next do iteracji

### Iteratory
Iterator sluzy do customowego chodzenia po kolekcji. Zwraca on przez yield return kolejne elementy.
Uzywajac slowa kluczowego yield okreslamy ze metoda, operator lub accessor get jest iteratorem.

In [9]:
static void Main()
{
    foreach (int number in SomeNumbers())
    {
        Console.Write(number.ToString() + " ");
    }
    // Output: 3 5 8
    Console.ReadKey();
}

public static System.Collections.IEnumerable SomeNumbers()
{
    yield return 3;
    yield return 5;
    yield return 8;
}

3 5 8 

Unhandled exception: System.InvalidOperationException: Cannot read keys when either application does not have a console or when console input has been redirected. Try Console.Read.
   at System.ConsolePal.ReadKey(Boolean intercept)
   at System.Console.ReadKey()
   at Submission#11.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

# Covariance i Contrvariance
Covariance in(contrvariance)-parametr moze uzyc bardziej derived klasy, out(covariance)-returnowac moze bardziej derived klase.
Stack thread: https://stackoverflow.com/questions/3445631/still-confused-about-covariance-and-contravariance-in-out

Uwaga kowariancja dziala tylko dla typów referencyjnych

In [ ]:
// The interface is covariant.
ICovariant<Button> ibutton = new SampleImplementation<Button>();
ICovariant<Object> iobj = ibutton;

// The class is invariant.
SampleImplementation<Button> button = new SampleImplementation<Button>();
// The following statement generates a compiler error
// because classes are invariant.
// SampleImplementation<Object> obj = button;

Dla kolekcji jest to przydatne na pryzkład w wypadku porównania: tworzymy wlasny comparer
 IEnumerable<Employee> noduplicates =  
            employees.Distinct<Employee>(new PersonComparer());  <-- uwaga w parametrze do distinct da sie podawac


Strong and weak typing:
weak typing no strong definition strongly typed lang can be dynamically typed!
! Note that in dynamically typed languages, values have types, not variables. Type inference: statyczne typowanie ktore domysla sie kompilator jak w Haskellu

### Expression trees
expression tree to tree like data structure gdize kazdy node reprezenture ekspresje typu operacja binarna lub wywolanie funkcji

Expression trees are also used in the dynamic language runtime (DLR) to provide interoperability between dynamic languages and .NET and to enable compiler writers to emit expression trees instead of Microsoft intermediate language (MSIL)

Drzewa ekspresji da sie wytwarzac z linqa lub recznie przez System.Linq.Expressions
Ekspresje można tworzyć z jednolinijkowych(tylko) lambd:

In [ ]:
Expression<Func<int, bool>> lambda = num => num < 5;  

Ekspresji używa się do Querying based on runtime state. Przykładem tego jest po prostu uzywanie IQueryable tak jak normlanie zapytania na bazie.
IQueryable ma dwa komponenty:
- Expression - a language- and datasource-agnostic representation of the current query's components, in the form of an expression tree.
- Provider - instancja providera LINQ który zna zasady materializacji query w wartości

Expression  trees mozna renderowac przy pomocy DebugView(todo zroobic program z tym)


Expression trees offer a way to inspect some piece of code before it is actually compiled, at runtime.

### Expressions vs delegates vs anomylous functions:
https://stackoverflow.com/questions/17709006/anonymous-methods-vs-lambda-expression
W przypadku gdyfunkcja spodziewa sie delegatu lambda jest skrótem do delegate{ ... }, jednak w przypadku Expressions<>
lambda jest przebudowywana w expression tree w celu pozniejszej optymalizacji.
Delegat daje tylkko metode do zbudowania, expression daje informacje co sie dzieje w srodku do czego np linq moze sie podpiac i optymalizowac na SQL.


### examples:
Few are the scenario's where you can use expression tree.

1) Dynamic Rule Engine - Rather than hard-coding the rule logic in your code. You can build your business-rule logic at run-time.

2) Dynamic Linq Queries - There are various scenario where developer need to write hard-coded queries. So you can eliminate this using expression tree.

## LINQ

Query expressions can be compiled to expression trees or to delegates, depending on the type that the query is applied to. IEnumerable<T> queries are compiled to delegates. IQueryable and IQueryable<T> queries are compiled to expression trees. For more information, see Expression trees.

Kiedy fluent exppression  typu lsit.Select a kiedy query expressino typu from list select .. ??
gdy robimy lety i joiny! se stacka O:
Neither is better: they serve different needs. Query syntax comes into its own when you want to leverage multiple range variables. This happens in three situations
- When using the let keyword
- When you have multiple generators (from clauses)
- When doing joins

W przypadku uzywania ArrayList trzeba podac typ!
### Linq grouping

In [ ]:
// custQuery is an IEnumerable<IGrouping<string, Customer>>
var custQuery =
    from cust in customers
    group cust by cust.City into custGroup
    where custGroup.Count() > 2
    orderby custGroup.Key
    select custGroup;

There are three separate ADO.NET (polaczenie odb z baza) Language-Integrated Query (LINQ) technologies: LINQ to DataSet, LINQ to SQL, and LINQ to Entities. LINQ to DataSet provides richer, optimized querying over the DataSet, LINQ to SQL enables you to directly query SQL Server database schemas, and LINQ to Entities allows you to query an Entity Data Model.

## assemblies
assembly to podstawoowa jednostka organizacyjna w .necie. asseembly sa implementowany jako .exe albo .dll

Kazdy komputer ktory ma CLR posiada Global Assembly Cache ktory przetrzymuje assemblies miedzy aplikacjami. Normalnie utrzymujemy assemlbies osobno
i wyszukujemy po lokalnym directory, niekioedy jednak mozemy odwolywac sie do <-- chyba tylko dotnet framework>

Assemblies can be static or dynamic. Static assemblies are stored on disk in portable executable (PE) files. Static assemblies can include interfaces, classes, and resources like bitmaps, JPEG files, and other resource files. You can also create dynamic assemblies, which are run directly from memory and aren't saved to disk before execution. You can save dynamic assemblies to disk after they have executed.

## Serialization
Json serialization noormalna serializacja mozna przy pomocy atrybutow i opcji zmieniac zachowanie.
Binarna serializacja- do kodu binarnego super szxbyka, niebezpieczna.


# Statements expressions and operators
typy statementów:
normalne: wybor z ifami, ekspresje, deklarace, lapanie wyjatkow, jump -break goto continue, iteracyjne,
Ciekawe:
- checked- normalnie kompilator wykryje overflow gdy dodamy dwie stale liczby, ale w przypadku gdy dodamy zmienna nie wykryje tego, uzywamy wtedy checked(duzaliczba+liczbaPowodujacaOverflow). Checked moze tez byc w blocku{}. Unchecked powoduje nie sprawdzanie przy dodawaniuliczb czy overflow.
- yield return - gdy iterator do tego dojdzie zwraca co tam jest, ale 'miejsce' w kodzie jest zapamietywane i egzekucja jest od neigo wznawiana.
Yield break przerywa iteracje.
- fixed- zapobiega GC relokacje zmiennej. Mozliwe tylkow  unsafe. N a przyklad robimy se zmienna refer i uzywamy wskaznika do niej. Fixed zabezpiieczy zeby wskaznik na nia wskazywal cnie
- label dla goto
- empty statement- przydatne! po prostu  mozna dac ; 
- lock- blokuje dostęp do kodu tlyko jednemu wątkowi jednocześnie

In [ ]:
//empty statement
void ProcessMessages()
{
    while (ProcessMessage())
        ; // Statement needed here.
}


//lock

lock (x)
{
    // Your code...
}

//wychodzi to samo co tutaj

object __lockObj = x;
bool __lockWasTaken = false;
try
{
    System.Threading.Monitor.Enter(__lockObj, ref __lockWasTaken);
    // Your code...
}
finally
{
    if (__lockWasTaken) System.Threading.Monitor.Exit(__lockObj);
}

### Reference i value equality

reference equality sprawdzamy System.Object.ReferenceEquals(a,b) i sprawdza to czy referujemy sie do tej samej zmiennej.


In [ ]:
  bool areEqual = System.Object.ReferenceEquals(a, b);
        // False:

Value equality dziala dla typow ze zdefiniowana metoda Equals i sprawdza wartosc typu. dl atypow prymitywnych jest to pproste dla klas trzeba samemu zdefiniowac interppretacje danych. Uwaga przy definiowaniu Equals powinno sie zeimplementowac je wedlug zasad matematycznych rownosci. Warto rowniez pobawid sie z gethashcode aby te same obiekty mialy taki sam hash! Uwaga reference equals nie powinno byc uzywane do stringow!

Uwaga od c#9 mozna dostac latwe porownanie za free przez recordy !

## Cast
Castowanie zachodzi w 4 sposoy:
- implicit
- explicit przy pomocy (typ)zmienna
- User definied 
- przy pomocy klas helper 

In [ ]:
//wlasna definicja casta
public readonly struct Digit
{
    private readonly byte digit;
    public static implicit operator byte(Digit d) => d.digit;
    public static explicit operator Digit(byte b) => new Digit(b);
}

**Is** operator:
Is testuje eskpresje czy jest danego typu, a od C#7 rowniez czy podaza za wzorem jak w haskellu. Uwaga nie dziala do lambd!

In [ ]:
object b = new Base();
        Console.WriteLine(b is Base);  // output: True
        Console.WriteLine(b is Derived);  // output: False


# Pattern matchning:
uwaga zeby uzywac tak jak w haskellu deklarujesz metode i swwitcha od niech w lambdzie
rodzaje matchingów:
- declaration and typa patterns: cos is string itp 
- constant type matching: w switcu po prostu 1=> cos, 2=> cos
- relational: >= 3 and < 6 => "spring",... porównania po prostu
- logical od C#9 można używac not and or w tych switchach
- property pattern: porownania w propertiesach: static bool IsConferenceDay(DateTime date) => date is { Year: 2020, Month: 5, Day: 19 or 20 or 21 }; - matchuje gdy rok jest taki jak podany miesiac tez i potem dzien jest lub lub. Mozna laczyc z logicznym i relacyjnym


In [ ]:
// declaration/type pattern matching
int i = 23;
object iBoxed = i;
int? jNullable = 7;
if (iBoxed is int a && jNullable is int b)
{
    Console.WriteLine(a + b);  // output 30
}

//pattern mathcing haskell style switch statement
public static decimal CalculateToll(this Vehicle vehicle) => vehicle switch
{
    Car => 2.00m,
    Truck => 7.50m,
    null => throw new ArgumentNullException(nameof(vehicle)),
    _ => throw new ArgumentException("Unknown type of a vehicle", nameof(vehicle)),
};

//relational patterns
static string Classify(double measurement) => measurement switch
{
    < -4.0 => "Too low",
    > 10.0 => "Too high",
    double.NaN => "Unknown",
    _ => "Acceptable",
};

Pattern mathcing with deconstruct

In [ ]:
public readonly struct Point
{
    public int X { get; }
    public int Y { get; }

    public Point(int x, int y) => (X, Y) = (x, y);

    public void Deconstruct(out int x, out int y) => (x, y) = (X, Y);
}

static string Classify(Point point) => point switch
{
    (0, 0) => "Origin",
    (1, 0) => "positive X basis end",
    (0, 1) => "positive Y basis end",
    _ => "Just a point",
};

zastępstwwem dla let? moze byc var z when 


In [ ]:
static Point Transform(Point point) => point switch
{
    var (x, y) when x < y => new Point(-x, y),
    var (x, y) when x > y => new Point(x, -y),
    var (x, y) => new Point(x, y),
};

W pattern matchingu _ reprezentuje cokolwiek i zazywa się discard patternem (w tym null)

Mozna rowniez matchowwac z nawiasem logicznie do

In [ ]:
if (input is not (float or double))
{
    return;
}

### AS casting
castowanie przy pomocy As korzysta z pattern matchingu 

The as operator considers only reference, nullable, boxing, and unboxing conversions. You can't use the as operator to perform a user-defined conversion. 
bez user defined!

In [ ]:
//as 
E as T
//to samo co 
E is T ? (T)(E) : (T)null

### boxing/unboxing
boxingiem nazywamy przypisanie do obiektu zmiennej niereferycjnej np:

In [ ]:
int i = 123;

// Boxing copies the value of i into object o.
object o = i;

Boxing polega na przesunieciu zmiennej na stos. Jest komputacyjnie kosztowe, wymaga zaalokowania na stosie pamięci dla nowego obiektu i jego przesunięcie tam. Some caveata przy unboxingu np 

In [ ]:
double e = 2.718281828459045;
object o = e; // box
int ee = (int)o; // runtime exception

### Byte arrays
LittleEdian

### dynamic
slowo kpluczowe dynamic overriduje statyczne typowanie c# i pozwala na typowanie w runtimie
dynamic d2 = "a string";

DLR umozliwil uzywanie dynamic w c#

# OOP

### Inheritance

New modifier excplicitly hides base class implementation of method. if new method is of the same name as method in base class new is assumed if not present. W przypadku **new** mozna wciaz sie dostac do bazowej metody skladnia bazowa.

In [ ]:
public class BaseC
{
    public int x;
    public void Invoke() { }
}
public class DerivedC : BaseC
{
    new public void Invoke() { }
}

override no override
The override modifier is required to extend or modify the abstract or virtual implementation of an inherited method, property, indexer, or event.

The overridden property must be virtual, abstract, or override.

In [ ]:
abstract class Shape
{
    public abstract int GetArea();
}

class Square : Shape
{
    //implementation
}
// Output: Area of the square = 144

https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/classes-and-structs/knowing-when-to-use-override-and-new-keywords
gdy implicit castujemy do klasy bazowej to przy new wykonamy metode klasy bazowej, przy override mimo ze jest typu bazowego wywola sie derived

### readonly
readonly fieldcan be assigned only in constructor

readonly **value** types are immuteable. Reference types moga siie odnosic tylko do tego samego obiektu
readolny struct-- struct jest imimutable
ref readonly methos- metoda zwraca refeerncje i write nie jest d

The readonly keyword is different from the const keyword. A const field can only be initialized at the declaration of the field. 

zasadnicza roznica: const jest compile time staly a readonly jest runtime staly
### abstract/virtual
virtual- derived class can override
abstract- any non-abtract derived class must override
### sealed
aby zapobiec dziedziczeniu metod pol po calej klasie dajemy sealed


### Finalizers (historically referred to as destructors) 
Finalizers cannot be defined in structs. They are only used with classes.
A class can only have one finalizer.
Finalizers cannot be inherited or overloaded.

The programmer has no control over when the finalizer is called; the garbage collector decides when to call it.
It's possible to force garbage collection by calling Collect, but most of the time, this call should be avoided because it may create performance issues.
when your application encapsulates unmanaged resources, such as windows, files, and network connections, you should use finalizers to free those resources

If your application is using an expensive external resource, we also recommend that you provide a way to explicitly release the resource before the garbage collector frees the object. To release the resource, implement a Dispose method from the IDisposable interface that performs the necessary cleanup for the object. This can considerably improve the performance of the application. Even with this explicit control over resources, the finalizer becomes a safeguard to clean up resources if the call to the Dispose method fails.

### dispose
Imlementujemy interfejs IDisposable. uwaga to jest lepsza metoda niz finalizer. finalezer dobry safeguarrd wtedy mozemy go supresowac

In [ ]:
public void Dispose()
{
    // Dispose of unmanaged resources.
    Dispose(true);
    // Suppress finalization.
    GC.SuppressFinalize(this);
}

### accebility keyword
- protected: jezeli klasa dziedziczy to moze miec access

A derived class has access to the public, protected, internal, and protected internal members of a base class. Even though a derived class inherits the private members of a base class, it cannot access those members. However, all those private members are still present in the derived class and can do the same work they would do in the base class itself. For example, suppose that a protected base class method accesses a private field. That field has to be present in the derived class in order for the inherited base class method to work properly.
Wiec private sa dziedziczone ale nie ma do nich dostepu.
- internal- kod w tym samym assembly ma do niego dostep
- private protected
- protected internal

W c# pola klas i struktur sa z defaultu private
Uwaga operatory musza byc zawsze public static.
interfejsy moga byc public/internal, public z defaultu i ich pola sa public.
delegaty sa z defaultu internal! i private gdy sa zagniezdzone

### field
Generally, you should use fields only for variables that have private or protected accessibility. Data that your type exposes to client code should be provided through methods, properties, and indexers. By using these constructs for indirect access to internal fields, you can guard against invalid input values.
Z defaultu uzywac propertiesow!
propertiesy moga byc abstract i wtedy mozemy poweidziec ze np bedzie tylko get

In [ ]:
 // Area is a read-only property - only a get accessor is needed:
    public abstract double Area
    {
        get;
    }

Side note: object initializers:

In [ ]:
Cat cat = new Cat { Age = 10, Name = "Fluffy" };

Od C#9 init accessor pozwwala na inicjalizacje readonly pol 

!!! propertiesy mozna wrzucac na interfesjach co jest ogromnym argumentem za ich uzywaniem

propertiesy mozna auto inicjalizowac:

In [ ]:
public string FirstName { get; set; } = "Jane";  

# Methods
passing by value vs by ref
Mozemy zwracac by reference

In [ ]:
public ref double GetEstimatedDistance()
{
    return ref estDistance;
}

//to store returned value we must declare ref local
ref int distance = Planet.GetEstimatedDistance();

## local functions
Local functions are explicitly named like methods. Lambda expressions are anonymous methods and need to be assigned to variables of a delegate type, typically either Action or Func types. When you declare a local function

Beginning with C# 10, some lambda expressions have a natural type, which enables the compiler to infer the return type and parameter types of the lambda expression.

If you declare a local function and only reference it by calling it like a method, it will not be converted to a delegate.
### variable capture
funkcje mozna wywolwac przed ich definicja i lapac zmienne z innego zaseigu

In [ ]:
int M()
{
    int y;
    LocalFunction();
    return y;

    void LocalFunction() => y = 0;
}

The instantiation necessary for lambda expressions means extra memory allocations, which may be a performance factor in time-critical code paths. Local functions do not incur this overhead. ????

w lambdach nie mozna uzywac yielda w locla functionach 

#### ref local
The return value must have a lifetime that extends beyond the execution of the method. In other words, it cannot be a local variable in the method that returns it.
jezeli przypiszemy Person p  bez slowa ref przekopiuje wartosci by value!

In [ ]:
ref Person p = ref contacts.GetContactInformation("Brandie", "Best");

A variable of a reference type does not contain its data directly; it contains a reference to its data. When you pass a reference-type parameter by value, it is possible to change the data belonging to the referenced object, such as the value of a class member. However, you cannot change the value of the reference itself

string is a reference type but it is passed by value
Strings are immutable!

a struct is a value type

### Extension methods
rextensiony sa statyczne!
When using an Onion Architecture or other layered application design, it's common to have a set of Domain Entities or Data Transfer Objects that can be used to communicate across application boundaries. These objects generally contain no functionality, or only minimal functionality that applies to all layers of the application. Extension methods can be used to add functionality that is specific to each application layer without loading the object down with methods not needed or wanted in other layers.

In [ ]:
public class DomainEntity
{
    public int Id { get; set; }
    public string FirstName { get; set; }
    public string LastName { get; set; }
}

static class DomainEntityExtensions
{
    static string FullName(this DomainEntity value)
        => $"{value.FirstName} {value.LastName}";
}

### named optional arguments

In [ ]:
public void ExampleMethod(int required, string optionalstr = "default string",
    int optionalint = 10)

## constructors
Static constructors - do inicjalizacji statystcznych pol

In [ ]:
public class Adult : Person
{
   private static int minimumAge;

   public Adult(string lastName, string firstName) : base(lastName, firstName)
   { }

   static Adult()
   {
      minimumAge = 18;
   }

   // Remaining implementation of Adult class.
}

Private constructor- A private constructor is a special instance constructor. It is generally used in classes that contain static members only. If a class has one or more private constructors and no public constructors, other classes (except nested classes) cannot create instances of this class.

Copy constructor- po prostu w parametrze obiekt tej samej klasy

## Partial classes
The partial modifier is not available on delegate or enumeration declarations.
At compile time, attributes of partial-type definitions are merged

All partial-type definitions meant to be parts of the same type must be defined in the same assembly and the same module

partial metody- deklaracje jak w .h w cpp

## Interfaces
oddball od c#8 mozna implementowac interfejsy ale implementacja jest dotepna tylko na interfejsie nie na implementujacej klasie:

In [ ]:
public interface IControl
{
    void Paint() => Console.WriteLine("Default Paint method");
}
public class SampleClass : IControl
{
    // Paint() is inherited from IControl.
}


var sample = new SampleClass();
//sample.Paint();// "Paint" isn't accessible.
var control = sample as IControl;
control.Paint();

## Delegates
A delegate is a type that represents references to methods with a particular parameter list and return type.

delegates can be multicasted to remove a method from the invocation list, use the subtraction or subtraction assignment operators (- or -=)

In [ ]:
var obj = new MethodClass();
Del d1 = obj.Method1;
Del d2 = obj.Method2;
Del d3 = DelegateMethod;

//Both types of assignment are valid.
Del allMethodsDelegate = d1 + d2;
allMethodsDelegate += d3;
//remove Method1
allMethodsDelegate -= d1;

// copy AllMethodsDelegate while removing d2
Del oneMethodDelegate = allMethodsDelegate - d2;

Beginning with C# 10, method groups with a single overload have a natural type. This means the compiler can infer the return type and parameter types for the delegate type:

In [ ]:
var read = Console.Read; // Just one overload; Func<int> inferred
var write = Console.Write; // ERROR: Multiple overloads, can't choose

## arrays
SomeType[] array4 = new SomeType[10];
dla typow referencyjnych to stworzy 10 nulli dla value stworzy 10 zmiennych typu sometype

- jagged array- tablica nierownomiernie wymiarowa

foreach moze zplaszczyc tablice:

In [ ]:
int[,] numbers2D = new int[3, 2] { { 9, 99 }, { 3, 33 }, { 5, 55 } };
// Or use the short form:
// int[,] numbers2D = { { 9, 99 }, { 3, 33 }, { 5, 55 } };

foreach (int i in numbers2D)
{
    System.Console.Write("{0} ", i);
}
// Output: 9 99 3 33 5 55

Using StringBuilder for Fast String Creation

The StringBuilder class creates a string buffer that offers better performance if your program performs many string manipulations. The StringBuilder string also enables you to reassign individual characters, something the built-in string data type does not support. 

## extern
dla importowanych metod

[DllImport("avifil32.dll")]
private static extern void AVIFileInit();

## eventy
public event EventHandler<ShapeEventArgs> ShapeChanged;
eventy robimy z event handlera generic jezeli mamy typ parametrow jezeli bez parametrow to po prostu albo przez dziedziczenie po event handlerze

## generics

constraints
https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/generics/constraints-on-type-parameters
- new() oznacza ze musi byc bezparametrowy konstruktor
- class oznacza ze musi byc typ referencyjny

## records
Not all data models work well with value equality. For example, Entity Framework Core depends on reference equality to ensure that it uses only one instance of an entity type for what is conceptually one entity. For this reason, record types aren't appropriate for use as entity types in Entity Framework Core.

# dedign patterns

### Abstract factory
abstract factory buduje rodizny boiektow ktore sa wariantami

In [ ]:
  public interface IAbstractFactory
    {
        IAbstractProductA CreateProductA();

        IAbstractProductB CreateProductB();
    }

### builder
builder trzyma za reke przy budowie 

Mozna uzyc z Director class jezeli automatyzowany jest proces budowy 

In [ ]:
 public interface IBuilder
    {
        void BuildPartA();
        
        void BuildPartB();
        
        void BuildPartC();
    }
// ... implementacja buildera ...
    public class Director
    {
        private IBuilder _builder;
        
        public IBuilder Builder
        {
            set { _builder = value; } 
        }
        
        // The Director can construct several product variations using the same
        // building steps.
        public void BuildMinimalViableProduct()
        {
            this._builder.BuildPartA();
        }
        
        public void BuildFullFeaturedProduct()
        {
            this._builder.BuildPartA();
            this._builder.BuildPartB();
            this._builder.BuildPartC();
        }
    }

### factory 
pryzklad: logistyka
Logistics obsluguje transport gdize wiekszosc logiki jest w klasie Truck. Chcemy dodac Ship. przesuwamy logike do klasy Logistics i ukrywamy tworzenie obiektu Truck Ship pod klasami dziedziczacymi roadLogistics i SeaLogistics ktore overriduja towrzneie obiektu ship/truck i dziela odziedziczona logike  
```
class Dialog is
    abstract method createButton():Button

     method render() is
        // Call the factory method to create a product object.
        Button okButton = createButton()
        // Now use the product.
        okButton.onClick(closeDialog)
        okButton.render()


class WindowsDialog extends Dialog is
    method createButton():Button is
        return new WindowsButton()

class WebDialog extends Dialog is
    method createButton():Button is
        return new HTMLButton()
interface Button is
    method render()
    method onClick(f)

// Concrete products provide various implementations of the
// product interface.
class WindowsButton implements Button is
    method render(a, b) is
        // Render a button in Windows style.
    method onClick(f) is
        // Bind a native OS click event.

class HTMLButton implements Button is
    method render(a, b) is
        // Return an HTML representation of a button.
    method onClick(f) is
        // Bind a web browser click event.

```

### Singleton
Singleton lajtowo ma isntacje ktora ma statyczny construcotr i statyczny instance trzyma jedyna isntacje go. Aby byl thread safe trzeba zrobić locka ktory jest statyczny i blokuje przy tworzeniu w konstruktorze singleton tak aby nie wytworzyl sie w wielu watkach razem.

### Prototype
To jest ICloneable w c#. Obiekt posiada wlasna deifnicje klonowania i implementuje interfejs ktorym mozemy sie poslugiwac.

## Structural

### Adapter

Tworzymy nowa klase Adapter ktora ma w srodku Adaptee i daje metode odslaniajaca cos w adaptee co wymaga opakowania.

### Bridge
Pryzdatny gdy korzystamy z  jakis ronzych DB/cloudow. Pozwala stworzyc abstrakcje nad implementacjami duzego obiektu i wymienic latwo implementacje. Generalnie cos takiego
```
class Abstraction
    {
        protected IImplementation _implementation;
        
        public Abstraction(IImplementation implementation)
        {
            this._implementation = implementation;
        }
        
        public virtual string Operation()
        {
            return "Abstract: Base operation with:\n" + 
                _implementation.OperationImplementation();
        }
    }
```
Bridge musi byc implementowany w poczatkowej fazie projekut

### Composite
Composite trzyma leafy ktore sa jego dziecmi w liscie. po prostu tree trzymajacy ponizsze moduly. Czesty przy gui.

### Decorator
Decorator dziedziczy po klasie ktora dekoruje i dodaje jakas funkcjonalnosc. Po prostu odslania metode przez override na metodzie i wywolaniu w componencie jej.

### Facade
Fasada Ukrywa skomplikowanie systemu w jednym obiekcie ograniczajacym api ale prostszym. Ukrywasz pare bibliotek pod tym i wywolujesz jedna operacje robioca tam duzo.

### Flyweight 
Flyweight oszczedza ram . Creation method zwraca zcacheowany obiekt zamiast tworzyc nowy.
Istnieje w niej shared state
### Proxy
tworzymy klasie imlementujaca interfejs serwisu ktory chcemy proxieowac robimy operacje i podajemy dalej.
W porownaniu do Adaptera podaje ten sam interfejs.

## Behavioral patterns
### Chain of Responsibility 
tworzymy interfejs handler ktory ustala next handler
i robimy cos w stylu

```
var monkey = new MonkeyHandler();
            var squirrel = new SquirrelHandler();
            var dog = new DogHandler();

            monkey.SetNext(squirrel).SetNext(dog);
```

### Command
Pakujemy operacje w obiekt o interfejsie z np execute.
Popularny w ui jako alternatywa dla callbackow i kolejkowaniu taskow, historii operacji.
### Iterator
No iterator idzie przez kolekcje cnie
### Mediator
Mediator reduces coupling between components
Mediator wystawia interfejs umozliwiajacy jego notifikacje, nastepnie wykonuje akcje bazujac na tej notyfikacji, zazwyczaj wywolujac metody na komponentach ktore zostaly mu przekazane w konstruktorze. Jest centralnym miejscem przekazujacym wiadomosci

### Memento 
umozliwia zapisywanie snapshotow obiektow. W c# rzadko uzywany lepiej robic serializacje.

### Observer
Observer ma metode Update, Subjectect ma attach na observer. To znaczy jezeli na subject jest next nastepny state to wsyzstkie observery so o tym poinformowane. 

### Strategy
Interfejs IStrategy wystawia metode bedaca logika, istnieje lilka jej implementacji.
istnieje metoda w klasie kontekstu ustawiajac astrategoie i kod tam se z niej korzysta.
### State
Jest jest uzywany zeby wywalic switche. State ma metode transition to ktora bierze kolejny state. Sam state ma rozne handlery ktore  obsluguja w zaleznosci od obecnego stanu.

### template
pozwala zdefiniowac jak wyglada mniej wiecej algorytmn i pozniej goimlementowac
Poozwala tworcom bibliotek pozostawic imlementacje dla uzytkonwika.

### visitor
nieuzywany pozwala rozszerzyc funkcjonalnosc hierarchii klas bez ich modyfikacji. skomplikowany